In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
from numba import jit

### Loading the dataset
This dataset is about predicting if a person makes more or less than 50,000 dollars

In [2]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = '.'
dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)

Let's train a classifier for this

In [3]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

Train 0.9350338780390594
Test 0.8489483747609943


### Getting an anchor

Now let's start the explainer. We need the training data to perturb instances.
`categorical_names` is a map from integer to list of strings, containing names for each
            value of the categorical features. Every feature that is not in
            this map will be considered as ordinal or continuous, and thus discretized.


In [4]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

Below, we get an anchor for prediction number 0. An anchor is a sufficient condition - that is, when the anchor holds, the prediction should be the same as the prediction for this instance.

In [5]:
def get_exp(idx):
    return explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

In [6]:
def get_fit_examples(exp, idx):
    return np.where(np.all(dataset.test[:, exp.features()] == dataset.test[idx][exp.features()], axis=1))[0]

In [7]:
def get_test_cov(fit_anchor):
    return (fit_anchor.shape[0] / float(dataset.test.shape[0]))

In [48]:
def compute_explenations(length=100):
    coverages = np.zeros(length)
    fit_examples = np.empty(shape = (length,), dtype = object)
    explenations = np.empty(shape = (length,), dtype = object)
    for i in range(length):
        if i % 5 == 0:
            print(i)
        cur_exp = get_exp(i)
        cur_fit = get_fit_examples(cur_exp, i)
        
        coverages[i] = get_test_cov(cur_fit)
        fit_examples[i] = cur_fit
        explenations[i] = cur_exp
    
    return coverages, fit_examples, explenations
                

In [49]:
cover, fit, explenations = best_k()

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


In [53]:
best = cover.argsort()[-10:]
for index in best:
    print("------------------------")
    exp = explenations[index]
    exp_label =  c.predict(dataset.test[index].reshape(1, -1))
    print('Anchor: %s' % (' AND '.join(exp.names())))
    print('Precision: %.2f' % exp.precision())
    print('Coverage: %.2f' % exp.coverage())
    print('Anchor test precision: %.2f' % (np.mean(c.predict(dataset.test[fit[index]]) == c.predict(dataset.test[index].reshape(1, -1)))))
    print('Anchor test coverage: %.2f' % (fit.shape[0] / float(dataset.test.shape[0])))
    covered_labels = dataset.labels_test[fit[index]]
    real_percentage = sum([1 if label == exp_label else 0 for label in covered_labels]) / len(covered_labels)
    print('Anchor test REAL precision: %.2f' % real_percentage)

------------------------
Anchor: Age <= 28.00 AND Marital Status = Never-married AND Relationship = Own-child
Precision: 0.99
Coverage: 0.08
Anchor test precision: 0.99
Anchor test coverage: 0.06
Anchor test REAL precision: 0.99
------------------------
Anchor: Age <= 28.00 AND Relationship = Own-child
Precision: 1.00
Coverage: 0.08
Anchor test precision: 0.99
Anchor test coverage: 0.06
Anchor test REAL precision: 0.99
------------------------
Anchor: Marital Status = Never-married AND Relationship = Own-child AND Capital Gain = 0
Precision: 0.96
Coverage: 0.09
Anchor test precision: 0.99
Anchor test coverage: 0.06
Anchor test REAL precision: 0.99
------------------------
Anchor: Age <= 28.00 AND Marital Status = Never-married AND Capital Gain = 0
Precision: 0.98
Coverage: 0.14
Anchor test precision: 0.97
Anchor test coverage: 0.06
Anchor test REAL precision: 0.97
------------------------
Anchor: Age <= 28.00 AND Marital Status = Never-married AND Capital Gain = 0
Precision: 0.99
Cover

In [5]:
idx = 0
np.random.seed(1)
print('Prediction: ', explainer.class_names[c.predict(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

Prediction:  b'>50K'


In [6]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: Education = Bachelors AND Relationship = Husband AND Marital Status = Married-civ-spouse AND Occupation = Sales AND Race = White
Precision: 0.96
Coverage: 0.02


Note that we set threshold to 0.95, so we guarantee (with high probability) that precision will be above 0.95 - that is, that predictions on instances where the anchor holds will be the same as the original prediction at least 95% of the time. Let's try it out on the test set

In [7]:
# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(dataset.test[:, exp.features()] == dataset.test[idx][exp.features()], axis=1))[0]
print('Anchor test precision: %.2f' % (np.mean(c.predict(dataset.test[fit_anchor]) == c.predict(dataset.test[idx].reshape(1, -1)))))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(dataset.test.shape[0])))

Anchor test precision: 0.97
Anchor test coverage: 0.02
